In [1]:
import uproot
from uproot_methods import TLorentzVectorArray as lv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
import pyro
import pyro.distributions as dist
import pyro.distributions.transforms as T
from pyro.nn import DenseNN, AutoRegressiveNN, ConditionalAutoRegressiveNN
from sklearn.preprocessing import StandardScaler

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    torch.cuda.empty_cache()
else:
    device = torch.device("cpu")
    torch.set_default_tensor_type("torch.FloatTensor")

In [2]:
# df  = pd.read_parquet('/home/taymaz/Documents/Flows/Data/data.kinematics.BDT.2018.parquet', engine='pyarrow')
# k = df[0:15000000]
# del df

df = pd.read_parquet('/home/taymaz/Documents/Flows/Data/data.kinematics.MDR_VEC.2018.parquet', engine='pyarrow', columns=['ntag', 'kinematic_region', 'pass_vbf_sel', 'm_h1', 'm_h2', 'log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh'])
print(len(df))
k = df[0:5500000]
del df

df = pd.read_parquet('/home/taymaz/Documents/Flows/Data/test_pred_2b_MDR_VEC.parquet', engine='pyarrow')
m = df[0:40000]
del df

#k = pd.read_hdf('/home/taymaz/Documents/Flows/Data/df_SM_2b_0.01_2b.h5')

46559280


In [3]:
# mask 2b/4b events & kinematic region & apply cuts
mask_2b     = k['ntag'] == 2
mask_4b     = k['ntag'] >= 4
mask_CR     = k['kinematic_region'] == 2 
mask_VR     = k['kinematic_region'] == 1 
mask_SR     = k['kinematic_region'] == 0 
mask_CRVR   = (k['kinematic_region'] == 1) | (k['kinematic_region'] == 2)
mask_vbf    = k['pass_vbf_sel'] == False

# train/test split
split   = 0.5                       # train/test split ratio
n       = int((1-split)*len(k))     # number of train samples

# train/test data
m_train = k.loc[(mask_2b & mask_CRVR & mask_vbf), ('m_h1', 'm_h2')][0:n]
k_train = k.loc[(mask_2b & mask_CRVR & mask_vbf), ('log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh')][0:n]
#m_test  = k.loc[(mask_2b & mask_CRVR & mask_vbf), ('m_h1', 'm_h2')][n+1:len(k)]
#k_test  = k.loc[(mask_2b & mask_CRVR & mask_vbf), ('log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh')][n+1:len(k)]
del k

# normalize
scaler_m_train = StandardScaler()
m_train = torch.tensor(scaler_m_train.fit_transform(m_train)).float()
scaler_k_train = StandardScaler()
k_train = torch.tensor(scaler_k_train.fit_transform(k_train)).float()

# mask SR in gaussian process data (for evaluation)
def SR(m1,m2):
    return np.sqrt(((m1-120)/(0.1*m1))**2+((m2-110)/(0.1*m2))**2)
mask_SR_gp = SR(m['m_h1'], m['m_h2']) < 1.6
m_eval = m.loc[(mask_SR_gp), ('m_h1', 'm_h2')]

# normalize gaussian process data
m_eval = torch.tensor(scaler_m_train.transform(m_eval)).float()
del m

In [4]:
# wrap variables and conditionals into pytorch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(Dataset, self).__init__()
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.X[idx], self.y[idx]

# prepare dataloaders
batch_size = 2**15
trainset = Dataset(k_train, m_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)
#testset = Dataset(k_test, m_test)
#testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [5]:
# flow model: see https://pyro.ai/examples/normalizing_flows_i.html
# can choose any flow from: https://github.com/pyro-ppl/pyro/tree/dev/pyro/distributions/transforms

# hyperparameters
hypernet_type = 'spline_autoregressive'     # mlp, resnet or autoregressive
num_flows     = 1                           # can stack multiple flows together
dim_m         = 2       
dim_k         = 5
count_bins    = 10                          # spline bins
bound         = 1.0                         # spline tail bound
order         = 'linear'                    # spline tail order
layers        = 5                           # number of hidden layers in mlp or residual block
blocks        = 2                           # number of residual blocks
hidden_dims_m = dim_m*10                    # for networks with input m
hidden_dims_k = dim_k*10                    # for networks with input k
dropout       = 0.1                         # dropout probability 
beta          = 1e-5                        # L2 regularization
lr            = 5e-3                        # adam learning rate
print_interval= 10                          # interval to print batch loss

# base distributions chosen to be gaussian (can change to distribution of choice)
dist_base_k = dist.Normal(torch.zeros(dim_k), torch.ones(dim_k), validate_args=False)

In [6]:
steps = 25         # total epochs
bootstrap = 0      # initialise bootstrap
bootstraps = 10    # number of re-trainings
    
while bootstrap < bootstraps:
    bootstrap = bootstrap + 1
        
    # Neural Autoregressive Flows
    if hypernet_type == 'affine_autoregressive':

        hypernet_conditional = ConditionalAutoRegressiveNN(
                                    input_dim=dim_k, 
                                    context_dim=dim_m, 
                                    hidden_dims=[hidden_dims_m]*layers, 
                                    skip_connections=False)

        # kinematic autoregressive transform: input_dim, hypernet, count_bins
        k_transform = [T.ConditionalAffineAutoregressive(hypernet_conditional) for _ in range(num_flows)]
        dist_k_given_m = dist.ConditionalTransformedDistribution(dist_base_k, k_transform)
    
    # Autoregressive Spline Flows
    elif hypernet_type == 'spline_autoregressive':

        hypernet_conditional = ConditionalAutoRegressiveNN(
                                    input_dim=dim_k, 
                                    context_dim=dim_m, 
                                    hidden_dims=[hidden_dims_m]*layers, 
                                    param_dims=[count_bins, count_bins, count_bins-1, count_bins] ,
                                    skip_connections=False)

        # kinematic autoregressive transform: input_dim, hypernet, count_bins
        k_transform = [T.ConditionalSplineAutoregressive(dim_k, hypernet_conditional, count_bins=count_bins) for _ in range(num_flows)]
        dist_k_given_m = dist.ConditionalTransformedDistribution(dist_base_k, k_transform)


    modules   = torch.nn.ModuleList(k_transform).to(device)   
    optimizer = torch.optim.Adam(modules.parameters(), lr=lr, weight_decay=beta)    
    
    for step in range(steps):
        
        running_train_loss = 0
        for batch, (k_batch, m_batch) in enumerate(trainloader):
            optimizer.zero_grad()
            
            # train log liklihood
            ln_p_k_given_m = dist_k_given_m.condition(m_batch).log_prob(k_batch)        
            loss_train = - ln_p_k_given_m.mean() 

            # take optimization step
            loss_train.backward()
            optimizer.step()

            dist_k_given_m.clear_cache()
            running_train_loss += loss_train.item()
            
            if batch % print_interval == print_interval-1:
                print('bootstrap: {}, epoch: {}, [{}/{} ({:.0f}%)], train loss: {:.3f}'.format(bootstrap, 
                                                                                       step, 
                                                                                       batch*batch_size, 
                                                                                       len(trainloader.dataset),
                                                                                       100.*batch/len(trainloader),
                                                                                       running_train_loss/print_interval))
                running_train_loss = 0

        # re-train if unstable
        if np.isnan(running_train_loss):
            bootstrap = bootstrap - 1
            print('Unstable training. Restarting bootstrap...')
            break
    
    # proceed to next bootstrap if stable   
    if not np.isnan(running_train_loss):

        # sample from transformed distribution
        k_flow_ = dist_k_given_m.condition(m_train).sample(torch.Size([len(m_train),]))

        # evaulate in SR
        k_eval_ = dist_k_given_m.condition(m_eval).sample(torch.Size([len(m_eval),]))

        # denormalize
        k_train_ = scaler_k_train.inverse_transform(k_train.cpu().detach().numpy())
        k_flow_  = scaler_k_train.inverse_transform(k_flow_.cpu().detach().numpy())
        k_eval_  = scaler_k_train.inverse_transform(k_eval_.cpu().detach().numpy())
        m_train_ = scaler_m_train.inverse_transform(m_train.cpu().detach().numpy())
        m_eval_  = scaler_m_train.inverse_transform(m_eval.cpu().detach().numpy())
        
        # convert to pandas
        k_train_ = pd.DataFrame(k_train_, columns = ['log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh'])
        k_flow_  = pd.DataFrame(k_flow_, columns = ['log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh'])
        k_eval_  = pd.DataFrame(k_eval_, columns = ['log_pT_h1', 'log_pT_h2', 'eta_h1', 'eta_h2', 'log_dphi_hh'])
        m_train_ = pd.DataFrame(m_train_, columns = ['m_h1', 'm_h2'])
        m_eval_  = pd.DataFrame(m_eval_, columns = ['m_h1', 'm_h2'])
        
        # undo preprocessing
        for i in ('log_pT_h1', 'log_pT_h2', 'log_dphi_hh'):
            k_flow_[i[4:]] = np.exp(k_flow_[i])
            k_train_[i[4:]] = np.exp(k_train_[i])
            k_eval_[i[4:]] = np.exp(k_eval_[i])
        k_flow_['dphi_hh'] = np.pi - k_flow_['dphi_hh']
        k_train_['dphi_hh'] = np.pi - k_train_['dphi_hh']
        k_eval_['dphi_hh'] = np.pi - k_eval_['dphi_hh']

        # find HC 4-vectors
        hc1_pred = lv.from_ptetaphim(k_flow_['pT_h1'], k_flow_['eta_h1'], np.zeros_like(k_flow_.index).astype(float), m_train_['m_h1'])
        hc2_pred = lv.from_ptetaphim(k_flow_['pT_h2'], k_flow_['eta_h2'], k_flow_['dphi_hh'], m_train_['m_h2'])
        hc1_pred_SR = lv.from_ptetaphim(k_eval_['pT_h1'], k_eval_['eta_h1'], np.zeros_like(k_eval_.index).astype(float), m_eval_['m_h1'])
        hc2_pred_SR = lv.from_ptetaphim(k_eval_['pT_h2'], k_eval_['eta_h2'], k_eval_['dphi_hh'], m_eval_['m_h2'])

        # boost into the di-higgs rest frame to get cos(theta*)
        hh_pred = hc1_pred + hc2_pred
        hh_pred_SR = hc1_pred_SR + hc2_pred_SR
        boost = - hh_pred.boostp3
        boost_SR = - hh_pred_SR.boostp3
        rest_hc1 = hc1_pred._to_cartesian().boost(boost)
        rest_hc1_SR = hc1_pred_SR._to_cartesian().boost(boost_SR)

        # calculate high level variables
        k_flow_['m_hh'] = hh_pred.mass
        k_flow_['m_hh_cor2'] = hh_pred.mass-hc1_pred.mass-hc2_pred.mass+250
        k_flow_['absCosThetaStar'] = np.abs(np.cos(rest_hc1.theta))
        k_flow_['pt_hh'] = hh_pred.pt
        k_eval_['m_hh'] = hh_pred_SR.mass
        k_eval_['m_hh_cor2'] = hh_pred_SR.mass-hc1_pred_SR.mass-hc2_pred_SR.mass+250
        k_eval_['absCosThetaStar'] = np.abs(np.cos(rest_hc1_SR.theta))
        k_eval_['pt_hh'] = hh_pred_SR.pt

        # append conditional variables
        k_flow_['m_h1'] = m_train_['m_h1']
        k_flow_['m_h2'] = m_train_['m_h2']
        k_eval_['m_h1'] = m_eval_['m_h1']
        k_eval_['m_h2'] = m_eval_['m_h2'] 

        # save results
        k_flow_.to_hdf('train_bootstrap_{}.h5'.format(bootstrap), key='df', mode='w')
        k_eval_.to_hdf('eval_bootstrap_{}.h5'.format(bootstrap), key='df', mode='w')




bootstrap: 1, epoch: 0, [294912/449026 (64%)], train loss: 7.208
bootstrap: 1, epoch: 1, [294912/449026 (64%)], train loss: 7.050
bootstrap: 1, epoch: 2, [294912/449026 (64%)], train loss: 6.983
bootstrap: 1, epoch: 3, [294912/449026 (64%)], train loss: 6.758
bootstrap: 1, epoch: 4, [294912/449026 (64%)], train loss: 6.358
bootstrap: 1, epoch: 5, [294912/449026 (64%)], train loss: 6.193
bootstrap: 1, epoch: 6, [294912/449026 (64%)], train loss: 6.120
bootstrap: 1, epoch: 7, [294912/449026 (64%)], train loss: 6.083
bootstrap: 1, epoch: 8, [294912/449026 (64%)], train loss: 6.112
bootstrap: 1, epoch: 9, [294912/449026 (64%)], train loss: 6.068
bootstrap: 1, epoch: 10, [294912/449026 (64%)], train loss: 6.046
bootstrap: 1, epoch: 11, [294912/449026 (64%)], train loss: 6.054
bootstrap: 1, epoch: 12, [294912/449026 (64%)], train loss: 6.066
bootstrap: 1, epoch: 13, [294912/449026 (64%)], train loss: 6.035
bootstrap: 1, epoch: 14, [294912/449026 (64%)], train loss: 6.021
bootstrap: 1, epoch:

KeyboardInterrupt: 